In [156]:
import music21
from tqdm import tqdm

In [59]:
def transpose_part(part, key):
    """Transposes a part to the key of C major."""
    transpose_interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch('C'))
    return part.transpose(transpose_interval)

In [ ]:
url = "https://kern.humdrum.org/cgi-bin/ksdata?location=musedata/mozart/quartet&file=k168-01.krn&format=kern"
music = music21.converter.parse(url)
freq_list = [ ([0] * 128) for _ in range(128) ]

for part in music.parts():
    key = part.recurse().getElementsByClass('Key')[0]
        
    transpose_interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch('C'))
    prev_midi = 0
    
    for elem in part.flatten():
        cur_midi = 0
        if isinstance(elem, music21.key.Key):
            key = elem
        if isinstance(elem, music21.note.Note):
            cur_midi = elem.transpose(transpose_interval).pitch.midi
        
        if prev_midi != 0 and cur_midi != 0:
            freq_list[prev_midi][cur_midi] += 1
            
        prev_midi = cur_midi
        
    print(freq_list[70])
            
            
    print(key)

In [127]:
def count_midi_frequencies(music, midi_freq_list):
    """
    Count the frequencies of transitions between MIDI pitches in `music`, and update 
    `midi_freq_list` with those counts. `midi_freq_list` should be a list of lists,
    where `midi_freq_list` is of length 128, and so are all of the lists at each of
    its indexes. Each index value represents the corresponding MIDI value.
    """
    for part in music.parts():
        key = part.recurse().getElementsByClass('Key')[0]

        transpose_interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch('C'))
        prev_midi = 0

        for elem in part.flatten():
            cur_midi = 0
            if isinstance(elem, music21.key.Key):
                key = elem
            if isinstance(elem, music21.note.Note):
                cur_midi = elem.transpose(transpose_interval).pitch.midi

            if prev_midi != 0 and cur_midi != 0:
                midi_freq_list[prev_midi][cur_midi] += 1

            prev_midi = cur_midi
    
    return midi_freq_list

In [159]:
def count_rhythm_frequencies(music, rhythm_freq_dict):
    for part in music.parts:
        for note in part.flatten().notes:
            sub_beat = note.beat % 1.0
            if sub_beat not in rhythm_freq_dict:
                rhythm_freq_dict[sub_beat] = {}
            
            length = float(note.quarterLength)
            if length not in rhythm_freq_dict[sub_beat]:
                rhythm_freq_dict[sub_beat][length] = 0
            rhythm_freq_dict[sub_beat][length] += 1
    
    return rhythm_freq_dict

In [160]:
midi_freq_list = [ ([0] * 128) for _ in range(128) ]
rhythm_freq_dict = {}

with open("kernscores_urls.txt") as file:
    url_list = file.read().splitlines()

for url in tqdm(url_list):
    music = music21.converter.parse(url)
    # count_midi_frequencies(music, midi_freq_list)
    count_rhythm_frequencies(music, rhythm_freq_dict)

100%|████████████████████████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.10s/it]


In [161]:
print(rhythm_freq_dict)

{0.0: {1.0: 1114, 1.5: 40, 0.5: 905, 0.25: 183, 0.75: 11, 3.0: 38, 0.16666666666666666: 10, 0.125: 4, 2.0: 252, 4.0: 32}, 0.5: {0.5: 895, 1.0: 11, 0.25: 250, 0.16666666666666666: 12, 0.125: 3, 0.08333333333333333: 6}, 0.75: {0.25: 252, 0.5: 4, 0.125: 12, 0.08333333333333333: 6}, 0.25: {0.25: 204, 0.5: 6, 0.125: 4}, 0.16666666666666674: {0.16666666666666666: 6}, 0.33333333333333326: {0.16666666666666666: 6}, 0.6666666666666667: {0.16666666666666666: 6, 0.08333333333333333: 6}, 0.8333333333333333: {0.16666666666666666: 6, 0.08333333333333333: 6}, 0.16666666666666652: {0.16666666666666666: 6}, 0.3333333333333335: {0.16666666666666666: 6}, 0.6666666666666665: {0.16666666666666666: 6}, 0.8333333333333335: {0.16666666666666666: 6}, 0.125: {0.125: 4}, 0.375: {0.125: 4}, 0.625: {0.125: 3}, 0.875: {0.125: 12}, 0.5833333333333333: {0.08333333333333333: 6}, 0.9166666666666667: {0.08333333333333333: 6}}
